In [1]:
import torch
import pickle
import os
from pathlib import Path
from config import get_config
from data_setup import train_test_dataloader
from model import build_model, Feature_Encoding

In [6]:
# loading checkpoint
path_to_model = Path(os.getcwd()).parents[0]/"data/unstacked/encoding_checkpoint_15_02_2350.pth"
checkpoint = torch.load(path_to_model)

In [7]:
## loading Model with checkpoint and extracting fetures from ReLU layer
config = get_config()
model = Feature_Encoding(**config["Encoding_parameters"])
model.load_state_dict(checkpoint["encoding"])

<All keys matched successfully>

In [35]:
# defining forward hook to extract features
features = {}
def getFeatures(layer = "activation_layer"):
    def hook(model, input, output):
        features[layer] = output.detach()
    return hook

In [16]:
## Extracting Features through last Layer -> ICA

In [27]:
h = model.activation_layer.register_forward_hook(getFeatures())

In [ ]:
# laoding train and test data

In [37]:
train_test_path = Path(os.getcwd()).parents[0]/"data/train_test_data.pkl"
with open(train_test_path, "rb") as f:
    train_test_data = pickle.load(f)

In [40]:
X_train = torch.tensor(train_test_data["train"]["X"])
X_test = torch.tensor(train_test_data["test"]["X"])
print(f"X Train: {X_train.shape} || X Test: {X_test.shape}")

X Train: torch.Size([8375, 440, 85]) || X Test: torch.Size([3589, 440, 85])


In [ ]:
# Extracting Features for Train data

In [44]:
_ = model(X_train)
X_train_features = features["activation_layer"]

In [ ]:
# Extracting Features from Test data

In [45]:
_ = model(X_test)
X_test_features = features["activation_layer"]

In [56]:
train_test_features = {
    "train": {
        "X" : X_train_features.numpy(),
        "y": train_test_data["train"]["y"]
    },
    "test":{
        "X" : X_test_features.numpy(),
        "y": train_test_data["test"]["y"]
    }
}

In [58]:
with open(Path(os.getcwd()).parents[0]/"data/train_test_features.pkl", "wb") as f:
    pickle.dump(train_test_features, f)